In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import re

In [3]:
df = pd.read_csv('result.csv')

In [4]:
df.head(10)

,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name
0,0,Clinical: Routine annual mammog...,28412120-45,BIRADS2,1.0,1.0,alcification|ass
1,1,CLINICAL: Screening~with no presenting sympto...,28408002-138,BIRADS3,1.0,1.0,alcification|ass
2,2,"CLINICAL: palpable mass left 2-3 o'clock,post ...",28413118-47,BIRADS6,1.0,1.0,alcification|ass
3,3,Clinical information: Screening ~ ~BILATERAL ...,28905280-118,BIRADS4B,1.0,NaN,alcification
4,4,CLINICAL: Screening~BILATERAL DIGITAL MAMMOGRA...,28883412-47,BIRADS2,1.0,1.0,alcification|ass
5,5,Clinical information:Screening~BILATERAL DIGIT...,28935614-58,BIRADS2,1.0,NaN,alcification
6,6,~CLINICAL: Follow ...,28854593-90,BIRADS3,1.0,1.0,alcification|ass
7,7,~CLINICAL: Follow ...,28854593-120,BIRADS3,1.0,1.0,alcification|ass
8,8,~CLINICAL: screenin...,22819148-48,BIRADS2,2.0,1.0,alcification|ass
9,9,Clinical information:Case right breast cancer ...,29451428-286,BIRADS2,0.0,0.0,NaN


In [5]:
df.Reports = df.Reports.apply(lambda x: x.replace('~','\n'))

In [6]:
result = [] 
no_pattern = "no.{0,80}calcif|No.{0,80}calcif"
no_change = "no.{0,20}change|No.{0,20}change"
calcification = 'calcif|calfication|oalcification'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows(): 
    if re.search(tomo_pattern,row[1]):
        result.append('Tomosynthesis')
        continue
    if re.search(calcification,row[1]):
        if re.search('\nMacrocalcification|\nMicrocalcification',row[1]):            
            row[1] = row[1].replace('\nMicrocalcification','\n')
            row[1] = row[1].replace('\nMacrocalcification','\n')
        allsentence = re.split('\\.|-|\n:',row[1])
        for iteration,sentence in enumerate(allsentence):                  
            if iteration < len(allsentence)-1:
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            result.append(1.0)
                            break
                    else :
                        continue
                elif re.search(calcification,sentence) :
                    result.append(1.0)
                    break
            else :
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            result.append(1.0)
                    else :
                        result.append(0.0)
                elif re.search(calcification,sentence) :
                    result.append(1.0)
                else :
                    result.append(0.0)
        
    else : 
        result.append(0.0)         
            

In [7]:
# only count for suspicious calcification 
resultcalc = [] 
no_pattern = "no.{0,80}calcif|No.{0,80}calcif|enign.{0,40}calcif"
no_change = "no.{0,20}change|No.{0,20}change"
calcification = 'morphous.{0,20}calcif|eterogeneous.{0,20}calcif|inear.{0,20}calcif|leomorph.{0,20}calcif|egment.{0,20}calcif|calc.{0,20}inside mass|roup.{0,10}(round|puncta){0,20}calcif'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows():
    if re.search(tomo_pattern,row[1]):
        resultcalc.append('Tomosynthesis')
        continue
    if re.search(calcification,row[1]):
        if re.search('\nMacrocalcification|\nMicrocalcification',row[1]):            
            row[1] = row[1].replace('\nMicrocalcification','\n')
            row[1] = row[1].replace('\nMacrocalcification','\n')
        allsentence = re.split('\\.|-|\n:',row[1])
        for iteration,sentence in enumerate(allsentence):                  
            if iteration < len(allsentence)-1:
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            resultcalc.append(1.0)
                            break
                            
                    else :
                        continue
                elif re.search(calcification,sentence) :
                    resultcalc.append(1.0)
                    break
            else :
                if re.search(no_pattern,sentence):
                    if re.search(no_change,sentence):
                        if re.search(calcification,sentence):
                            resultcalc.append(1.0)
                    else :
                        resultcalc.append(0.0)
                elif re.search(calcification,sentence) :
                    resultcalc.append(1.0)
                else :
                    resultcalc.append(0.0)
        
    else : 
        resultcalc.append(0.0)         
            

In [8]:
print(df.Reports.loc[57])

STEREOTACTIC GUIDED CORE NEEDLE BIOPSY:
PREVIOUS MAMMOGRAMS: 
      - Low suspicion for malignancy of two groups of mild pleomorphic microcalcifications at superficial right upper outer quadrant.
 
PROCEDURE:  Routine stereotactic core needle biopsy was performed under local anesthesia and sterile technique.
      Six core specimens were obtained.
      All of core specimens contain microcalcifications.
      There was some incisional bleeding.
      The study was stopped.
      Specimens were fixed in 10% formalin and sent for pathology.
IMPRESSION: 
      CORE NEEDLE BIOPSY SUCESSFULLY PERFORMED.
      Suggest to appointment with breast surgeon when pathology result complete.



In [9]:
resultmass = [] 
remove_pattern = 'obscure.{0,10}\n.{0,15}(nodule|mass)|obscure.{0,60}(mass|odule)|\nMass|\nSolid mass|(linical|LINICAL|P\.E\.).{0,60}(mass|odule)|uggest breast self examination.*?\n'
no_pattern = 'no.{0,60}mass|No.{0,60}mass|without.{0,60}mass'
no_change_pattern = "no.{0,20}change|No.{0,20}change"
ultrasound_pattern = 'Ultrasound|ULTRASOUND|ultrasound|SONOGRA|Sonogra|sonogra'
mass_pattern = 'mass|nodu'
previous_pattern = 'Previous.{0,20}|previous.{0.20}'
recommendation_pattern = 'Recom|RECOM|Reccom|RECCOM|recom'
tomo_pattern = 'Tomosynthesis|TOMOSYNTHESIS|tomosynthesis|omogra|OMOGRA'
for index,row in df.iterrows():
    if re.search(tomo_pattern,row[1]):
        resultmass.append('Tomosynthesis')
        continue
    # remove false positive pattern 
    if re.search(remove_pattern,row[1]):
        row[1]=re.sub(remove_pattern,'\n',row[1])
    # remove recommendation part and note
    if re.split(recommendation_pattern,row[1]) == 2:
        row[1] = re.split(recommendation_pattern,row[1])[0]
    #find ultrasound in report if in row 1-4 - dont use that report to interpret
    if re.search(ultrasound_pattern,row[1]):
        # if there is ultrasound in first 4 line of report
        if len(re.findall('\n',re.split(ultrasound_pattern,row[1])[0]))<=3 :
            sentencein4line = re.split('\\.|-|\n:',re.split(ultrasound_pattern,row[1])[0])
            # look whether there is 'previous' in last line before splited by ultrasoud or not
            if re.search(previous_pattern,sentencein4line[-1]):
                row[1]= re.split(ultrasound_pattern,row[1])[1]                
            else :
                # if there is no seperate part of MMG and ultrasound - append nan
                if len(re.split(ultrasound_pattern,row[1])) == 2:
                    resultmass.append(np.nan)
                    continue
                # if there seperate part of MMG and ULT , use the second part 
                else :
                    row[1]= re.split(ultrasound_pattern,row[1])[1]  
        else :
            row[1]= re.split(ultrasound_pattern,row[1])[0]
    allsentence = re.split('\\.|-|\n:',row[1])
    for iteration,sentence in enumerate(allsentence): 
        if iteration < len(allsentence)-1:
            if re.search(mass_pattern,sentence):
                if re.search(no_pattern,sentence):
                    if re.search(no_change_pattern,sentence):
                        resultmass.append(1.0)
                        break
                    else :    
                        continue
                else :
                    resultmass.append(1.0)
                    break
        else :
            if re.search(mass_pattern,sentence):
                if re.search(no_pattern,sentence):
                    resultmass.append(0.0)
                else :
                    resultmass.append(1.0)
            else :
                resultmass.append(0.0)

In [10]:
len(resultmass)

15603

In [12]:
resultmass[250]

1.0

In [13]:
print(df.Reports.loc[250])

Clinical information:Follow up study
BILATERAL DIGITAL MAMMOGRAPHY:
Technique:standard CC, MLO views
Comparison study:to 31 Aug 2018  
Breast density      Heterogeneously dense fibroglandular tissue with could may obscures detection of small masses.
Mass  No significant change over two years of a circumscribed isodense mass at left upper outer quadrant
MicrocalcificationsNot detectable
Macrocalcification  Macrocalcifications in both breasts
Architectural distortionNot detectable
Focal asymmetry/Focal densityNot detectable
Dilated tubular structureNot detectable
Skin thickeningNot detectable
Associated findings:Not detectable
HANDHELD ULTRASOUND:
Breast tissue composition      Heterogeneous background
Cystic massNot detectable
Solid mass  Stable of a 5x4.7x6.5 mm oval hypoechoic nodule without vascularity at left upper outer quadrant (1 o'clock)
Architectural distortionNot detectable
Associated findings-Benign appearing axillary lymph nodes, bilaterally.
   - Two calcific foci at right 

In [14]:
df.head()

,Unnamed: 0,Reports,ACC No.,BIRADS,Mmg Calcification,Mmg Mass,Disease Name
0,0,Clinical: Routine annual mammog...,28412120-45,BIRADS2,1.0,1.0,alcification|ass
1,1,CLINICAL: Screening\nwith no presenting sympt...,28408002-138,BIRADS3,1.0,1.0,alcification|ass
2,2,"CLINICAL: palpable mass left 2-3 o'clock,post ...",28413118-47,BIRADS6,1.0,1.0,alcification|ass
3,3,Clinical information: Screening \n \nBILATERA...,28905280-118,BIRADS4B,1.0,NaN,alcification
4,4,CLINICAL: Screening\nBILATERAL DIGITAL MAMMOGR...,28883412-47,BIRADS2,1.0,1.0,alcification|ass


In [15]:
df['All_calcification']=result
df['Suspicious_calcification'] = resultcalc
df['Mass'] = resultmass

In [16]:
df.to_csv('GT.csv')